In [1]:
import numpy as np
import pandas as pd


# URL
# https://waterdata.usgs.gov/pa/nwis/uv?format=rdb&site_no=01571500&period=&begin_date=2021-01-01&end_date=2021-01-20


In [10]:
BASE_URL = 'https://waterdata.usgs.gov/pa/nwis/uv'
FORMAT = 'rdb' # rdb
SITE_NUMBER = '01571500' # Yellow Breeches Creek near Camp Hill, PA
BEGIN_DATE = '2020-01-01' # 1985-10-01 / 2007-10-01
END_DATE = '2020-12-31' # 2021-01-18 / 2021-01-18

In [14]:
# url_ = BASE_URL + '?' + 'format=' + FORMAT + '&site_no=' + SITE_NUMBER + '&period=&begin_date=' + BEGIN_DATE + '&end_date=' + END_DATE
url_ = 'https://nwis.waterdata.usgs.gov/pa/nwis/uv/?format=rdb&site_no=01571500&period=&begin_date=2015-01-01&end_date=2020-12-31'
import requests
import json
response = requests.get(url_).text
# response

file1 = open("data/USGS/Yellow-Breeches-Creek/raw.txt","w")#write mode 
file1.write(response) 
file1.close() 

In [19]:
csv = pd.read_csv('data/USGS/Yellow-Breeches-Creek/20150101-20201231.csv', skiprows=30)
csv.head()

,agency_cd,site_no,datetime,tz_cd,122083_00065,122083_00065_cd,122084_00060,122084_00060_cd
0,USGS,1571500,1/1/15 0:00,EST,1.37,A,223.0,A
1,USGS,1571500,1/1/15 0:15,EST,1.38,A,228.0,A
2,USGS,1571500,1/1/15 0:30,EST,1.37,A,223.0,A
3,USGS,1571500,1/1/15 0:45,EST,1.37,A,223.0,A
4,USGS,1571500,1/1/15 1:00,EST,1.37,A,223.0,A
